In [1]:
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from numba import jit
import math

import xgboost as xgb

from libs.extract import extract_url
from libs.metrics import smape_fast
from libs.data_engineering import calculerLag

In [3]:
size_data = 100
lag = 8

In [4]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:size_data]
train_1_reduce = train_1_reduce.fillna(0)
df_extract = extract_url(train_1_reduce['Page'])
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
df_lag = calculerLag(train_1_reduce,lag).reset_index()
data = df_lag.set_index('Page').join(df_extract.set_index('Page'))
data = data.reset_index().set_index(['Dates','Page']).sort_index()

In [5]:
data.head(9)

diff1  diff2  \
Dates      Page                                                               
2015-07-01 2NE1_zh.wikipedia.org_all-access_spider               NaN    NaN   
           2PM_zh.wikipedia.org_all-access_spider                NaN    NaN   
           3C_zh.wikipedia.org_all-access_spider                 NaN    NaN   
           4minute_zh.wikipedia.org_all-access_spider            NaN    NaN   
           52_Hz_I_Love_You_zh.wikipedia.org_all-access_sp...    NaN    NaN   
           5566_zh.wikipedia.org_all-access_spider               NaN    NaN   
           91Days_zh.wikipedia.org_all-access_spider             NaN    NaN   
           A'N'D_zh.wikipedia.org_all-access_spider              NaN    NaN   
           AKB48_zh.wikipedia.org_all-access_spider              NaN    NaN   

                                                               diff3  diff4  \
Dates      Page                                                               
2015-07-01 2NE1_zh.wikipedia.org_all-access_spider               NaN    NaN   
           2PM_zh.wikipedia.org_all-access_spider                NaN    NaN   
           3C_zh.wikipedia.org_all-access_spider                 NaN    NaN   
           4minute_zh.wikipedia.org_all-access_spider            NaN    NaN   
           52_Hz_I_Love_You_zh.wikipedia.org_all-access_sp...    NaN    NaN   
           5566_zh.wikipedia.org_all-access_spider               NaN    NaN   
           91Days_zh.wikipedia.org_all-access_spider             NaN    NaN   
           A'N'D_zh.wikipedia.org_all-access_spider              NaN    NaN   
           AKB48_zh.wikipedia.org_all-access_spider              NaN    NaN   

                                                               diff5  diff6  \
Dates      Page                                                               
2015-07-01 2NE1_zh.wikipedia.org_all-access_spider               NaN    NaN   
           2PM_zh.wikipedia.org_all-access_spider                NaN    NaN   
           3C_zh.wikipedia.org_all-access_spider                 NaN    NaN   
           4minute_zh.wikipedia.org_all-access_spider            NaN    NaN   
           52_Hz_I_Love_You_zh.wikipedia.org_all-access_sp...    NaN    NaN   
           5566_zh.wikipedia.org_all-access_spider               NaN    NaN   
           91Days_zh.wikipedia.org_all-access_spider             NaN    NaN   
           A'N'D_zh.wikipedia.org_all-access_spider              NaN    NaN   
           AKB48_zh.wikipedia.org_all-access_spider              NaN    NaN   

                                                               diff7  diff8  \
Dates      Page                                                               
2015-07-01 2NE1_zh.wikipedia.org_all-access_spider               NaN    NaN   
           2PM_zh.wikipedia.org_all-access_spider                NaN    NaN   
           3C_zh.wikipedia.org_all-access_spider                 NaN    NaN   
           4minute_zh.wikipedia.org_all-access_spider            NaN    NaN   
           52_Hz_I_Love_You_zh.wikipedia.org_all-access_sp...    NaN    NaN   
           5566_zh.wikipedia.org_all-access_spider               NaN    NaN   
           91Days_zh.wikipedia.org_all-access_spider             NaN    NaN   
           A'N'D_zh.wikipedia.org_all-access_spider              NaN    NaN   
           AKB48_zh.wikipedia.org_all-access_spider              NaN    NaN   

                                                               Visitors  \
Dates      Page                                                           
2015-07-01 2NE1_zh.wikipedia.org_all-access_spider                 18.0   
           2PM_zh.wikipedia.org_all-access_spider                  11.0   
           3C_zh.wikipedia.org_all-access_spider                    1.0   
           4minute_zh.wikipedia.org_all-access_spider              35.0   
           52_Hz_I_Love_You_zh.wikipedia.org_all-access_sp...       0.0   
           5566_zh.wikipedia.org_all-access_spider        

In [6]:
data = pd.get_dummies(data)

In [26]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE','MAPE'])

X = data.drop('Visitors', axis = 1).values
y = data['Visitors'].values

tscv = TimeSeriesSplit()
print(tscv)

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    param = {'max_depth':2, 'eta':1, 'objective=reg' :' tweedie'}
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    smape = smape_fast(y_test, preds)
    mape = np.mean(np.abs(preds - y_test)/np.abs(y_test))
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    print("Test MAPE: %.3f" % mape)
    new_metrics = pd.DataFrame([(rmse,smape,mape)], columns = ['RMSE','SMAPE','MAPE'])
    print(new_metrics)
    metrics_df = metrics_df.append(new_metrics, ignore_index=True)

TimeSeriesSplit(max_train_size=None, n_splits=5)
TRAIN: [   0    1    2 ... 9167 9168 9169] TEST: [ 9170  9171  9172 ... 18333 18334 18335]
[17:31:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 28.137
Test SMAPE: 96.907
Test MAPE: inf
        RMSE      SMAPE  MAPE
0  28.136762  96.907153   inf
TRAIN: [    0     1     2 ... 18333 18334 18335] TEST: [18336 18337 18338 ... 27499 27500 27501]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


[17:31:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 20.168
Test SMAPE: 90.621
Test MAPE: inf
        RMSE      SMAPE  MAPE
0  20.168189  90.621494   inf
TRAIN: [    0     1     2 ... 27499 27500 27501] TEST: [27502 27503 27504 ... 36665 36666 36667]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


[17:31:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 21.030
Test SMAPE: 78.249
Test MAPE: inf
        RMSE      SMAPE  MAPE
0  21.030072  78.249365   inf
TRAIN: [    0     1     2 ... 36665 36666 36667] TEST: [36668 36669 36670 ... 45831 45832 45833]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


[17:31:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 28.742
Test SMAPE: 65.071
Test MAPE: inf
        RMSE      SMAPE  MAPE
0  28.741687  65.070559   inf
TRAIN: [    0     1     2 ... 45831 45832 45833] TEST: [45834 45835 45836 ... 54997 54998 54999]
[17:31:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 20.910
Test SMAPE: 64.066
Test MAPE: inf
        RMSE      SMAPE  MAPE
0  20.909741  64.065587   inf


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in true_divide


In [27]:
metrics_df

,RMSE,SMAPE,MAPE
0,28.136762,96.907153,inf
1,20.168189,90.621494,inf
2,21.030072,78.249365,inf
3,28.741687,65.070559,inf
4,20.909741,64.065587,inf


In [28]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()
moyenne_MAPE = metrics_df['MAPE'].mean()

In [29]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)
print("Test moyenne MAPE: %.3f" % moyenne_MAPE)

Test moyenne RMSE: 23.797
Test somme RMSE: 118.986
Test moyenne SMAPE: 78.983
Test moyenne MAPE: inf
